In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import time
to_timestamp = lambda strdate: time.mktime(time.strptime(str(strdate), '%Y-%m-%d %H:%M:%S'))

# Load and split data

In [2]:
path = '../../data/raw/Датасет3_Дтп.xlsx'
dataset_name = path.split('/')[-1][:-5]

df = pd.read_excel(path, index_col='Время регистрации', parse_dates=True)
df = df[(df['Широта']!=0) & (df['Долгота']!=0)]
print(df.shape)
df.head(3)

(21493, 6)


,Категория,Идентификатор Еас адреса,Идентификатор Еас здания,Широта,Долгота,Район
Время регистрации,,,,,,
2019-01-01 01:01:13,ДТП с пострадавшими людьми,NaN,NaN,59.937400,30.282000,Василеостровский
2019-01-01 01:01:49,ДТП с пострадавшими людьми,NaN,197864.0,59.936924,30.282853,Василеостровский
2019-01-01 01:01:53,ДТП с пострадавшими людьми,NaN,93814.0,59.946515,30.271703,Василеостровский


In [8]:
df.tail(3)

,ab,Идентификатор Еас адреса,Идентификатор Еас здания,Широта,Долгота,Район
Время регистрации,,,,,,
2020-08-12 22:11:33,ДТП с пострадавшими людьми,45734.0,36499.0,59.949145,30.348193,Центральный
2020-08-12 23:01:38,ДТП с пострадавшими людьми,448817.0,130108.0,59.834779,30.372879,Московский
2020-08-12 23:57:54,ДТП с пострадавшими людьми,104037.0,82898.0,59.746600,30.581741,Колпинский


In [ ]:
# 2019-01-01 - 2020-08-12

In [8]:
train = df.loc['2019-01-01 01:01:13':'2020-03-01']
test = df.loc['2020-02-02':'2020-08-12']
len(train), len(test)

(16601, 5936)

# Feature data

In [ ]:
class WeekDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        timestamps = list(map(to_timestamp, df.index.tolist()))
        x = sorted(list(set(timestamps)))
        y = [timestamps.count(t) for t in x]
        
        
        
    def scale(self, x, y, k=60):
        newx, newy = [], []
        frm = int(min(x))
        to  = int(max(x))
        i = 0
        for s in range(frm, to, k):
            while x[i] < s:
                i += 1
            j = i
            while x[j] < min(to, s+k):
                j += 1
            vals = y[i:j]
            newx.append(len(newx))
            newy.append(sum(vals))
        return newx, newy